In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import sys
sys.path.append('..')

from data_center.binance_handler import BinanceHandHandler
from backtest.performance_generater import *
from backtest.factor_analysis_tool import FactorAnalysisTool
from backtest.operators import *

import alphalens
import quantstats as qs
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### 使用的是日資料(時區是Asia/Hong_Kong)，也就是說我要在亞洲時區的00:00:00進場

In [2]:
Open = BinanceHandHandler().get_factor_data('open')
High = BinanceHandHandler().get_factor_data('high')
Low  = BinanceHandHandler().get_factor_data('low')
Close = BinanceHandHandler().get_factor_data('close')
Volume = BinanceHandHandler().get_factor_data('volume')
Volvalue = BinanceHandHandler().get_factor_data('volvalue')
Takerbuy = BinanceHandHandler().get_factor_data('takerbuy')
Takerbuyvalue = BinanceHandHandler().get_factor_data('takerbuyvalue')

returns = Close.pct_change()
expreturn = Close.pct_change().shift(-2)
Benchmark = expreturn['BTCUSDT']

### 要打的標的

In [3]:
Close.columns.to_list()

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'AAVEUSDT',
 'AVAXUSDT',
 'ETCUSDT',
 'SOLUSDT',
 'ADAUSDT',
 'LINKUSDT',
 'BCHUSDT',
 'DOTUSDT',
 'LTCUSDT',
 'UNIUSDT',
 'XRPUSDT',
 'XLMUSDT',
 'MATICUSDT',
 'TRXUSDT',
 'DOGEUSDT',
 'INJUSDT',
 'DASHUSDT',
 'XEMUSDT',
 'ZECUSDT']

### 因子運算方法

In [4]:
# sharpe factor
period = 21
rolling_returns = returns.rolling(window = period).mean()
rolling_std_dev = returns.rolling(window = period).std()
sharpe_factor = rolling_returns / rolling_std_dev

# sum_returns_factor
sum_ret_factor = returns.rolling(window=20).sum()

# bias factor
def ts_sma(df: pd.DataFrame, d: int):
    sma = df.apply(lambda x: ta.sma(x, length=d), axis=0)
    return sma

def ts_ema(df: pd.DataFrame, d: int):
    ema = df.apply(lambda x: ta.ema(x, length=d), axis=0)
    return ema

def ts_wma(df: pd.DataFrame, d: int) -> pd.DataFrame:
    weights = np.arange(1, d + 1)
    
    def wma(x):
        return np.dot(x, weights) / weights.sum()
    wma_df = df.rolling(window=d, min_periods=d).apply(wma, raw=True)
    
    return wma_df
def ts_bias(df, d, method='sma'):
    if method == 'sma':
        ma = ts_sma(df, d)
    elif method == 'ema':
        ma = ts_ema(df, d)
    elif method == 'wma':
        ma = ts_wma(df, d)
    else:
        raise ValueError("Invalid method. Please use 'sma', 'ema', or 'wma'.")

    bias = (df - ma) / ma * 100
    return bias

bias_factor = ts_bias(df = Close, d = 27, method='sma')


# cmo factor
def ts_cmo(df: pd.DataFrame, d: int) -> pd.DataFrame:

    delta = df.diff()
    
    up = delta.clip(lower=0) 
    down = -delta.clip(upper=0)  

    sum_up = up.rolling(window=d, min_periods=d).sum()
    sum_down = down.rolling(window=d, min_periods=d).sum()

    cmo = (sum_up - sum_down) / (sum_up + sum_down) * 100
    return cmo

cmo_factor = ts_cmo(Close, 20)

# combine together and only give signal to quantile five and one
Multi_alpha = sharpe_factor + sum_ret_factor + bias_factor + cmo_factor
ranked_df = Multi_alpha.dropna().rank(axis=1, pct=True)
signal_df = ranked_df.where(~((ranked_df > 0.2) & (ranked_df < 0.8)), np.nan)

### 將訊號轉換成 weighting (我需要把30%的weighting 放在BTC現貨)

In [5]:
demean = signal_df.sub(signal_df.mean(axis=1), axis=0)
weighting = demean.div(demean.abs().sum(axis=1), axis=0)
factor_weighting = weighting*0.7

btc_weighting = pd.DataFrame(0.3, index=factor_weighting.index, columns=['BTCUSDT_SPOT'])

pool_weighting = pd.concat([factor_weighting, btc_weighting], axis=1)
pool_weighting

,BTCUSDT,ETHUSDT,BNBUSDT,AAVEUSDT,AVAXUSDT,ETCUSDT,SOLUSDT,ADAUSDT,LINKUSDT,BCHUSDT,...,XRPUSDT,XLMUSDT,MATICUSDT,TRXUSDT,DOGEUSDT,INJUSDT,DASHUSDT,XEMUSDT,ZECUSDT,BTCUSDT_SPOT
datetime,,,,,,,,,,,,,,,,,,,,,
2022-09-12,NaN,NaN,NaN,NaN,-0.092,0.079,0.070,0.052,0.061,NaN,...,NaN,NaN,0.088,-0.074,-0.101,NaN,NaN,NaN,-0.083,0.3
2022-09-13,NaN,NaN,NaN,NaN,-0.101,0.070,NaN,0.061,0.052,NaN,...,NaN,NaN,0.079,NaN,-0.092,NaN,NaN,NaN,-0.083,0.3
2022-09-14,NaN,NaN,-0.074,NaN,-0.101,0.088,NaN,0.052,0.070,NaN,...,NaN,NaN,0.061,NaN,-0.092,NaN,NaN,-0.083,NaN,0.3
2022-09-15,NaN,NaN,NaN,-0.074,-0.101,0.052,NaN,0.079,0.088,NaN,...,NaN,NaN,0.070,NaN,-0.092,NaN,NaN,-0.083,NaN,0.3
2022-09-16,NaN,-0.074,NaN,NaN,-0.101,NaN,NaN,0.061,0.088,NaN,...,0.079,NaN,NaN,NaN,NaN,0.07,NaN,-0.083,-0.092,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-25,NaN,NaN,0.079,NaN,0.070,-0.074,0.088,-0.092,NaN,0.061,...,NaN,NaN,NaN,-0.101,0.052,NaN,NaN,NaN,NaN,0.3
2024-03-26,NaN,NaN,0.079,NaN,0.070,-0.083,0.088,-0.074,NaN,0.061,...,NaN,NaN,NaN,-0.101,0.052,NaN,NaN,NaN,NaN,0.3
2024-03-27,NaN,NaN,0.061,NaN,0.052,-0.083,0.079,-0.074,NaN,0.088,...,NaN,NaN,NaN,-0.101,0.070,NaN,NaN,NaN,NaN,0.3


In [10]:
pool_weighting[-1:]

,BTCUSDT,ETHUSDT,BNBUSDT,AAVEUSDT,AVAXUSDT,ETCUSDT,SOLUSDT,ADAUSDT,LINKUSDT,BCHUSDT,...,XRPUSDT,XLMUSDT,MATICUSDT,TRXUSDT,DOGEUSDT,INJUSDT,DASHUSDT,XEMUSDT,ZECUSDT,BTCUSDT_SPOT
datetime,,,,,,,,,,,,,,,,,,,,,
2024-03-29,NaN,-0.083,0.052,NaN,NaN,NaN,0.079,NaN,NaN,0.088,...,NaN,NaN,-0.092,-0.101,0.07,NaN,NaN,NaN,NaN,0.3
